In [10]:
#Tarea 2: Codigo para limpiar y appendear las bases de matching de ambas rondas
#Ultima actualizacion: 09/Ago/2022

# Appendea las bases de cada ronda
# Dropea variables irrelevantes
# Me quedo con una respuesta por publicacion
#### 1) Dropeo los mails de confirmación no matcheados
#### 2) Me quedo con 1 match por publicación
#### 3) Me quedo con 1 rta por thred_id

#### REVISAR
#Inputs: (A) base de scraps de properati (randomization) y (B) base de mails con respuestas, en las cuales haya confirmaciones de properati
#Outputs: (A) sent_mails_conf_envio_[FECHA], que agrupa las publicaciones a las que les enviamos correctamente el mail, y (B) not_sent_mails_conf_envio_[FECHA]_[PAIS] que muestra, para cada país, las inmobiliarias a las que les tenemos que reenviar los correos.


### TODO:
# - generar id inmobiliaria (hash)
# - 

In [21]:
# Importar paquetes necesarios

import pandas as pd
import numpy as np
from datetime import datetime
import tools

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

pd.options.mode.chained_assignment = None  # default='warn'

#### Ronda y fecha de cada ronda
fecha_round_1_matching = datetime.strptime('2022_08_11', "%Y_%m_%d")
fecha_round_2_matching = datetime.strptime('2022_08_11', "%Y_%m_%d")
fecha_comienzo_round_2 = datetime.strptime('2022_05_03', "%Y_%m_%d")

#### Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)
#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"
path_dropbox = r"C:\Users\oficina825\Dropbox"

#Rutas Luis 
#path_dropbox = r"C:\Users\37615965\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#### Rutas utilizadas (autogeneradas, no tocar):
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_dataout = path_dropbox + r"\BID-LGBTQ+\campo\data\para_analisis"

#### Clave de ids
identifica_respuestas  = ['thread_id_inmob','fecha','from_inmob']
identifica_publicacion = ['link','municipio','id','ronda']

In [22]:
#Importar bases con los matches completos
# df = pd.read_excel(path_dataout + fr"\base_limpia_tarea_2_nico_sin_RA.xlsx") # Base sin matches corregidos manualmente
df = pd.read_excel(path_dataout + fr'\df_final_con_matches_manuales_RA_r1_y_r2.xlsx')

In [23]:
df.pais.value_counts()

arg    2134
col     550
per     315
ecu     294
Name: pais, dtype: int64

In [24]:
df[['fecha','date_inmob_clean']]

,fecha,date_inmob_clean
0,2021-11-23,2022-04-21 11:04:56
1,2022-03-03,2022-04-20 14:39:34
2,2022-04-18,2022-04-19 13:37:41
3,2022-03-17,NaT
4,2021-11-26,2022-04-19 13:29:41
...,...,...
8673,2022-03-17,2022-05-14 11:56:22
8674,2022-03-17,2022-05-14 11:56:22
8675,2022-03-17,2022-05-14 11:56:22
8676,2022-04-11,2022-05-04 22:01:56


In [25]:
### Limpio variable tiempo
# df[['date_inmob_clean','date_conf_clean']] = tools.limpia_variable_dates(df,['date_inmob','date_conf'])

### Limpia prueba google y quien sos
df = tools.limpia_prueba_google_y_quien_sos(df, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')

###### Variables a Dropear

# Indices: En esta lista se incorporan todos los indices residuales que fueron quedando en las distintas etapas del merge
lista_unnameds_e_indexes = [col for col in df.columns if 'Unnamed' in col or 'index' in col] 

# Variables que no varían...
vars_irrelevantes = []  # Voy completando con el loop
for col in df.columns:
    if len(df[col].dropna().unique()) == 1: # Dropeo las que solo tiene uno o dos valores (hay varias con un solo valor + missing, y son totalmente irrelevantes)
        vars_irrelevantes = vars_irrelevantes + [col]

# Otras variables irrelevantes
vars_otras = ['caracteristicas','script_string_3','cc_conf','thread_id_conf','index_scrap_conf','match_string_conf','match2_string_conf','index_scrap_inmob'] 

to_drop = [col for col in df.columns if col in lista_unnameds_e_indexes + vars_irrelevantes + vars_otras]
print('Variables a dropear:\n',to_drop, sep='')

######### Drop variables
# Verificar antes de correrlo... Podría haber algo que me perdí!
df.drop(columns=to_drop, inplace=True)

# Limpio duplicados
pre = len(df)
df = df[-df.fillna(0).duplicated(keep='first')] # Dropeo los duplicados (uso fillna ya que los nan python no los considera iguales)
print("Se dropearon", pre-len(df), "observaciones duplicadas") # FIXME: chequear esto

Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 0 de ¿Quien sos?
Variables a dropear:
['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 4']
Se dropearon 76 observaciones duplicadas


In [26]:
pd.crosstab(df.date_inmob_clean.isna(), df.tipo)

tipo,match_ambas,solo_publicacion,solo_respuesta
date_inmob_clean,,,
False,4294,0,179
True,0,4129,0


In [27]:
print(df.tipo.isna().sum())
print(df.ronda.isna().sum())

0
0


In [28]:
#### Dropeo los mails de confirmación no matcheados
print("Verificación (tiene que dar 0):", "\nHay", len(df[df['tipo']=='']), "confirmaciones de respuesta que no matchearon con nada")

## Genero la diferencia entre la confirmacion y la respuesta:
df['time_diff'] = df['date_inmob_clean'] - df['date_conf_clean']
df['time_diff'] = df['time_diff'].dt.total_seconds() / 60

### Construyo orden de las respuestas
df['mail_number'] = tools.genera_orden_de_respuestas(df, ['from_inmob','id_publicacion'], 'date_inmob')

print('Me quedarían',len(df),'observaciones en la base')
df.tipo.value_counts()

Verificación (tiene que dar 0): 
Hay 0 confirmaciones de respuesta que no matchearon con nada
Me quedarían 8602 observaciones en la base


match_ambas         4294
solo_publicacion    4129
solo_respuesta       179
Name: tipo, dtype: int64

In [29]:
pd.crosstab(df['RA'],df['tipo'])

tipo,match_ambas
RA,
anaguadalupedepasqua@hotmail.com,210
analiariveroborges@gmail.com,178
canuelobelen@gmail.com,210
castano.fernando.martin@gmail.com,201
euge.basile.eb@gmail.com,83
euge.basile.eb@gmail.com,68
manuelfmb12@gmail.com,190
marianomenendez0@gmail.com,90
marianomenendez0@gmail.com,71


In [30]:
pd.crosstab(df.date_inmob_clean.isna(), df.tipo)


tipo,match_ambas,solo_publicacion,solo_respuesta
date_inmob_clean,,,
False,4294,0,179
True,0,4129,0


In [31]:
pd.crosstab(df.tipo,df.ronda)

ronda,1,2
tipo,,
match_ambas,2122,2172
solo_publicacion,2072,2057
solo_respuesta,95,84


In [32]:
df[(df['ronda']==2) & (df['tipo']=='solo_respuesta')]

,ronda,tipo,id_publicacion,link,id,nombre,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,subject_inmob,from_inmob,to_inmob,body_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,id_pub2,RA_inmob_name,RA_match_comment,time_diff,uno,order,mail_number
4649,2,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,arg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,f04ca5358f0f69823459a6f5c97c941cdf645b99271f5a...,"Pablo, te enviamos un mensaje",Feldman Propiedades <noresponder@adinco.net>,gome.zpablo.u@gmail.com,[<p>Adinco *Feldman Propie...,"Fri, 06 May 2022 13:09:42 +0000",NaN,180997d8abb009a0,multipart/alternative,NaN,2022-05-06,2022-05-06 10:09:42,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
3946,2,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,arg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,af31f7b12ccaddaa575b6c120436baf1d84e7172fac33c...,respuesta a su consulta,Federico semeria <semeriafed@gmail.com>,j.fmanuelrodriguez@gmail.com,[<p>Hola Manuel-\r\n\r\naca les dejo los datos...,"Fri, 6 May 2022 14:08:16 -0300",NaN,1809a582261abcdb,multipart/alternative,NaN,2022-05-06,2022-05-06 14:08:16,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
5437,2,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,col,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,7ab4740885b3e9953d5530ee0fcf3a7ac3116e1f29e0b9...,RENTA APARTAMENTO AMOBLADO NORTE BOGOTA,MARY LYAN <marylyan@hotmail.com>,josed.rodriguez.w@gmail.com,[<p>Buenas tardes José y Luz\r\n\r\nMe presen...,"Fri, 6 May 2022 20:24:08 +0000",NaN,1809b0b5292e9140,multipart/alternative,NaN,2022-05-06,2022-05-06 17:24:08,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
4601,2,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,arg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,a579b7eca777e8b61699de240176a21d9b12db2fb445cf...,¡OPORTUNIDAD para vos!,"""comunicaciones@melamudpropiedades.com"" <venta...",gomezfh.pa.blo@gmail.com,[<p>Su Cliente de Mail NO soporta mensajes en ...,"Mon, 9 May 2022 11:57:53 -0300 (-03)",NaN,180a95ac4c1e4a60,multipart/alternative,NaN,2022-05-09,2022-05-09 11:57:53,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
6593,2,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,per,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,3000b99270d4841ba6720f29c26618a79dbadc69da6480...,Respuesta a tu consulta - zoegrupoinmobiliari,zoegrupoinmobiliari - Contacto <b059bc9e9373c2...,j.danielflores.n@gmail.com,[<p>@import url(http://url2214.inmomail.com/ls...,"Mon, 09 May 2022 04:20:02 +0000 (UTC)",NaN,180a70bb5c871bec1,multipart/alternative,NaN,2022-05-09,2022-05-09 01:20:02,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [33]:
##### Matcheo con la base de RA - round 1
import importlib
importlib.reload(tools)

# Levanta la base
ra1 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round1-messages April 26\Agostina_mensajesra1_2nd_rev1.xlsx")
ra1['ra'] = 'Agostina'
ra1 = ra1.rename(columns={'Observaciones':'observacion'}) 
ra2 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round1-messages April 26\Alejo_mensajesra3_2nd_rev1.xlsx")
ra2['ra'] = 'Alejo'
ra3 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round1-messages April 26\Julian_mensajesra3_2nd_rev1.xlsx")
ra3['ra'] = 'Julian'

ra_list = [ra1,ra2,ra3]
for data in ra_list:
    data.columns = [x.lower().rstrip() for x in data.columns]

ra = pd.concat(ra_list)

identifica_respuestas  = ['thread_id','fecha','from']
ra = tools.limpia_prueba_google_y_quien_sos(ra, path_dropbox, from_name='from', subject_name='subject')
ra['fecha'] = tools.limpia_variable_dates(ra,['date'])
ra['id_respuesta'] = tools.genera_id(ra, identifica_respuestas)

# Elimino respuestas de prueba y duplicados... ¿Por qué hay duplicados?
ra = ra[ra.duplicated(subset=['thread_id','date','from','body'], keep='first')==False]
ra = ra[ra['categoria']!=0] # Categoría 0 agrupa todos los mails inválidos

Se dropearon 6 observaciones de Google.
Se dropearon 191 observaciones ('Julian Test').
Se dropearon 5 observaciones 'de prueba'.
Se dropearon 7 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 0 de ¿Quien sos?


In [34]:
##### Matcheo con la base de RA - round 2 + post 25

ra1 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round2\Agostina_mensajesra1_2dap_revision1.xlsx")
ra1['ra'] = 'Agostina'
ra2 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round2\Alejo_mensajesra3_2dap_revision1.xlsx")
ra2['ra'] = 'Alejo'
ra3 = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round2\Julian_mensajesra2_2dap_revision1.xlsx")
ra3['ra'] = 'Julian'

ra_list = [ra1,ra2,ra3]
for data in ra_list:
    data.columns = [x.lower().rstrip() for x in data.columns]

ra_round2 = pd.concat(ra_list)

identifica_respuestas  = ['thread_id','fecha','from']
ra_round2 = tools.limpia_prueba_google_y_quien_sos(ra_round2, path_dropbox, from_name='from', subject_name='subject')
ra_round2['fecha'] = tools.limpia_variable_dates(ra_round2,['date'])
ra_round2['id_respuesta'] = tools.genera_id(ra_round2, identifica_respuestas)


# Elimino respuestas de prueba y duplicados... ¿Por qué hay duplicados?
ra_round2 = ra_round2[ra_round2.duplicated(subset=['thread_id','date','from','body'], keep='first')==False]
# ra_round2['ronda'] = np.where(ra_round2['date_clean']<fecha_comienzo_round_2,1,2)
ra_round2 = ra_round2[ra_round2['categoria']!=0] # Categoría 0 agrupa todos los mails inválidos... TENEMOS ESTO HECHO DOS VECES. PODRIAMOS VERIFICARLOS!!!

Se dropearon 20 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 144 observaciones 'de prueba'.
Se dropearon 461 observaciones de mails nuestros.
Se dropearon 64 observaciones de MDS.
Se dropearon 143 de ¿Quien sos?


In [35]:
merged_test = ra_round2.merge(df, on='id_respuesta', how='left', indicator=True)
print(merged_test._merge.value_counts())

####################################################
# FIXME: OJO CON ESTAS 107 OBSERVACIONES DE LA RONDA 2 QUE DESAPARECIERON EN ALGUN MOMENTO. REVISAR
# Posiblemente hayan desaparecido en los matches automáticos. Verificar.
####################################################
test = merged_test[merged_test['_merge']=='left_only'][['thread_id','fecha_x','from']]

both          2087
left_only      107
right_only       0
Name: _merge, dtype: int64


In [36]:
# test = merged_test[(merged_test['_merge']=='left_only')]
# test.to_excel(r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\para_analisis\bases auxiliares\107no_matcheados.xlsx")

In [37]:
data = merged_test[merged_test['_merge']=='left_only'][['from','thread_id']]
for mail_from, mail_ti in data.to_records(index=False):
    filtered = df[(df['from_inmob']==mail_from) & (df['thread_id_inmob']==mail_ti)]
    if len(filtered)>0:
        print("La observación", df[df['from_inmob']==mail_from].index, "de la base df contiene el mail", mail_from)

    else:
        print("La observación de", mail_from, "no se encuentra en df")

La observación de Solimano Dal Pos <info@solimanodalpos.com.ar> no se encuentra en df
La observación de patriciaerhardt1@gmail.com no se encuentra en df
La observación de Eliana Avila <eliana@avilas.com.ar> no se encuentra en df
La observación de Francisco Marciano <ventas@victoriacordeiro.com> no se encuentra en df
La observación de "Belén Fuentes" <mb@fuentesyfuentes.com.ar> no se encuentra en df
La observación de MULTIHOUSE INMOBILIARIA - Contacto <d057645c9b5d526031e6edc8cb862b77_0.14331.0.993632.rsp7954813@inmomail.com> no se encuentra en df
La observación de Rosemary Diaz <rosemary@cuno.com.ar> no se encuentra en df
La observación de Constructora Inmobiliaria Gallego Asociados <notificaciones@domus.la> no se encuentra en df
La observación de andujarbel@gmail.com no se encuentra en df
La observación de Ivan Dario Toro Garcia <ivantoro@prointegral.com> no se encuentra en df
La observación de Tizado Propiedades <caballito@tizado.com> no se encuentra en df
La observación de Adriana V

In [38]:
merged_test[merged_test['_merge']=='left_only']

,subject,from,body,date,cc,thread_id,mimetype,pais_x,categoria,colateral,telefono,automatico,no es,nombre_x,observaciones,ra,columna3,observacion,rta,date_clean,fecha_x,id_respuesta,ronda,tipo,id_publicacion,link,id,nombre_y,inmobiliaria,fecha_y,municipio,ciudad,pais_y,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,subject_inmob,from_inmob,to_inmob,body_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,id_pub2,RA_inmob_name,RA_match_comment,time_diff,uno,order,mail_number,_merge
19,Casas A ESTRENAR en Barrio Privado,Solimano Dal Pos <info@solimanodalpos.com.ar>,[<p>Visita la versión online para ver el conte...,"Tue, 3 May 2022 11:05:02 +0000 (UTC)",NaN,180899855a25094a,multipart/alternative,arg,2.0,NaN,NaN,6.0,NaN,NaN,NaN,Agostina,NaN,NaN,NaN,2022-05-03 08:05:02,2022-05-03 08:05:02,eaee6027ef36010f44e60b4138ecbf11a31a6f23e55c94...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,"#########, novedades de Godoy Cruz 2875",patriciaerhardt1@gmail.com,"[<p>Hola #########, Respecto de esta propiedad...","Fri, 6 May 2022 05:08:29 -0700",NaN,180994589cbbca2c,multipart/alternative,arg,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Agostina,NaN,NaN,NaN,2022-05-06 09:08:29,2022-05-06 09:08:29,56adc8a8a731689d8ac72af98888cc60ffa6ae60271768...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26,Hemos encontrado nuevas propiedades para tu bú...,Eliana Avila <eliana@avilas.com.ar>,[<p>Listado de propiedades adjuntas.</p>],"Fri, 06 May 2022 12:43:00 -0000",NaN,1809965181bc43de,multipart/alternative,arg,2.0,NaN,NaN,2.0,NaN,NaN,promo,Agostina,NaN,NaN,NaN,2022-05-06 09:43:00,2022-05-06 09:43:00,5bfd2c702807e7843246157cd4700470532f7ea9356102...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
87,Cambio de precio - Viamonte al 1400,Francisco Marciano <ventas@victoriacordeiro.com>,[<p>######### cómo estas? Te cuento que la pro...,"Sat, 7 May 2022 05:47:58 -0700",NaN,1809e900e90e17e6,multipart/alternative,arg,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Agostina,NaN,NaN,NaN,2022-05-07 09:47:58,2022-05-07 09:47:58,eba74a8de1e9749a213613fa559a8738fb0c3716e460aa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
91,Detalles de la propiedad - Alvear Icon,"""Belén Fuentes"" <mb@fuentesyfuentes.com.ar>","[<style id=""media-query-bodytag"" type=""text/cs...",5 May 2022 15:19:24 -0300,NaN,180

In [39]:
merged_test['ronda'] = np.where(merged_test['date_clean']<fecha_comienzo_round_2,1,2)

pd.crosstab(merged_test.ronda, merged_test._merge)

_merge,left_only,both
ronda,,
1,5,196
2,102,1891


In [40]:
# Mergeo la base con las clasificaciones de los RA
ra = ra.rename(columns={'no es':'noes', 'from':'from_inmob', 'subject':'subject_inmob','body':'body_inmob'})
ra_identifies = ['id_respuesta']
ra_variables = ['categoria','colateral','telefono','automatico','noes','nombre','observacion','ra','from_inmob','subject_inmob','body_inmob']
master = df.merge(ra[ra_variables + ra_identifies], on=ra_identifies, how='left',indicator=True)
print(master._merge.value_counts())

# Agrego data del body, from y subject de las 11 respuestas que desaparecieron en la ronda 1
master = tools.normaliza_columnas_post_merge(master,suffixes=('_x','_y'))

# Agrego data de la ronda y el tipo para los que no mergearon 
master['ronda'] = np.where(master['_merge']=='right_only',1,master['ronda'])
master['tipo'] = np.where(master['_merge']=='right_only','solo_respuesta',master['tipo'])
master['categorizado_pero_no_matcheado'] = np.where(master['_merge']=='right_only',True,False)

# Dropeo las columnas irrelevantes
master = master.drop(columns=['_merge'])


left_only     6664
both          1938
right_only       0
Name: _merge, dtype: int64
Hay valores diferentes en las columans nombre_y y nombre_x


,nombre_x,nombre_y
0,departamento en alquiler en panamericana km 40...,Fermin
2,departamento en alquiler en estudios wyndham 0,#########lia Chavez
71,departamento en alquiler en san miguel,sigifredo rosales
73,apartamento en arriendo en chia,Sonia
75,departamento en alquiler en corrientes 300,######### Cardoso
...,...,...
7705,departamento en alquiler en miraflores,Alexandra Delgado
7706,departamento en alquiler en san miguel,Carmen Villaverde
7707,departamento en alquiler en miraflores,
7708,departamento en alquiler en miraflores,######### Requena


Hay valores diferentes en las columans subject_inmob_y y subject_inmob_x


,subject_inmob_x,subject_inmob_y
78,"Manuel, necesito tu teléfono!","#########, necesito tu teléfono!"
80,Fwd: Nueva solicitud para Apartamento - Alquil...,Fwd: Nueva solicitud para Apartamento - Alquil...
96,"Manuel, te enviamos un mensaje","#########, te enviamos un mensaje"
102,"Carlos, propiedades que te gustarán ❤","#########, propiedades que te gustarán ❤"
112,"Pablo, te enviamos un mensaje","#########, te enviamos un mensaje"
...,...,...
7704,Re: Nueva solicitud para Departamento - Alquil...,Re: Nueva solicitud para Departamento - Alquil...
7707,Re: Nueva solicitud para Departamento - Alquil...,Re: Nueva solicitud para Departamento - Alquil...
7708,"Penthouse de Av Benavides, Miraflores","Penthouse de Av Benavides, Mira#########"
7709,Penthouse en Miraflores,Penthouse en Mira#########


Hay valores diferentes en las columans body_inmob_y y body_inmob_x


,body_inmob_x,body_inmob_y
2,"[<center>\n<div class=""banner"">\n<div class=""d...","[<center>\n<div class=""banner"">\n<div class=""d..."
71,"[<meta content=""text/html; charset=utf-8"" http...","[<meta content=""text/html; charset=utf-8"" http..."
72,[<p>Buenas noches_x000D_\nCuando quieren ir a ...,[<p>Buenas noches\nCuando quieren ir a verlo? ...
73,"[<meta content=""text/html; charset=utf-8"" http...","[<meta content=""text/html; charset=utf-8"" http..."
74,[<p>@import url(http://url2214.inmomail.com/ls...,[<p>@import url(http://url2214.inmomail.com/ls...
...,...,...
7706,[<p>Estimado me comunico con ud porque dejó un...,[<p>Estimado me comunico con ud porque dejó un...
7707,[<p>Su número de contacto pf._x000D_\n_x000D_\...,[<p>Su número de contacto pf._x000D_\n_x000D_\...
7708,[<p>Muy buenas tardes saludos cordiales coordi...,[<p>Muy buenas tardes saludos cordiales coordi...
7709,"[<p>Estimado Señor Daniel, nos gustaría atende...","[<p>Estimado Señor #########, nos gustaría ate..."


In [41]:
pd.crosstab(master.date_inmob_clean.isna(), master.tipo)


tipo,match_ambas,solo_publicacion,solo_respuesta
date_inmob_clean,,,
False,4294,0,179
True,0,4129,0


In [42]:
# Mergeo la base con las clasificaciones de los RA ronda 2
ra_round2 = ra_round2.rename(columns={'no es':'noes', 'from':'from_inmob', 'subject':'subject_inmob','body':'body_inmob'})
ra_identifies = ['id_respuesta']
ra_variables = ['categoria','colateral','telefono','automatico','noes','nombre','observacion','ra','from_inmob','subject_inmob','body_inmob']
master = master.merge(ra_round2[ra_variables + ra_identifies], on=ra_identifies, how='left',indicator=True)
print(master._merge.value_counts())

master = tools.normaliza_columnas_post_merge(master,suffixes=('_x','_y')) # VERIFICAR

# Agrego data de la ronda y el tipo para los que no mergearon 
master['ronda'] = np.where(master['_merge']=='right_only',2,master['ronda'])
master['tipo'] = np.where(master['_merge']=='right_only','solo_respuesta',master['tipo'])
master['categorizado_pero_no_matcheado'] = np.where(master['_merge']=='right_only',True,master['categorizado_pero_no_matcheado'])

# Dropeo las columnas irrelevantes
master = master.drop(columns=['_merge'])

left_only     6517
both          2087
right_only       0
Name: _merge, dtype: int64
Hay valores diferentes en las columans nombre_y y nombre_x


,nombre_x,nombre_y
4,departamento en alquiler en san antonio 1800,Melanie
10,departamento en alquiler en la carolina,Alexandra
14,departamento en alquiler en guayaquil,ingryd
21,departamento en alquiler en aguero y santa fe ...,Elisabeth mañanes
23,departamento en alquiler en balconcillo,Katherine
...,...,...
8592,departamento en alquiler en miraflores,Beatriz Mora
8594,departamento en alquiler en miraflores,beatriz
8598,departamento en alquiler en san borja,anthony
8602,departamento en alquiler en san isidro,Requena


Hay valores diferentes en las columans subject_inmob_y y subject_inmob_x


,subject_inmob_x,subject_inmob_y
18,"Manuel, te enviamos un mensaje","#########, te enviamos un mensaje"
19,"Manuel, te enviamos un mensaje","#########, te enviamos un mensaje"
20,"Pablo, te enviamos un mensaje","#########, te enviamos un mensaje"
27,"Pablo, te enviamos un mensaje","#########, te enviamos un mensaje"
33,Respuesta a tu consulta - carlos alexis bautis...,Respuesta a tu consulta - ######### alexis bau...
...,...,...
8577,Re: Nueva solicitud para Departamento - Alquil...,Re: Nueva solicitud para Departamento - Alquil...
8581,"Dpto en Alquiler, Miraflores","Dpto en Alquiler, Mira#########"
8594,Alquiler de dpto segundo piso con cochera en M...,Alquiler de dpto segundo piso con cochera en M...
8597,Re: Nueva solicitud para Departamento - Alquil...,Re: Nueva solicitud para Departamento - Alquil...


Hay valores diferentes en las columans body_inmob_y y body_inmob_x


,body_inmob_x,body_inmob_y
3,"[<style id=""media-query-bodytag"" type=""text/cs...","[<style id=""media-query-bodytag"" type=""text/cs..."
4,"[<center>\n<div class=""banner"">\n<div class=""d...","[<center>\n<div class=""banner"">\n<div class=""d..."
5,"[<div>\nBuenas tardes Manuel, la unidad se enc...","[<div>\nBuenas tardes #########, la unidad se ..."
6,"[<center>\n<div class=""banner"">\n<div class=""d...","[<center>\n<div class=""banner"">\n<div class=""d..."
9,"[<meta content=""text/html; charset=utf-8"" http...","[<meta content=""text/html; charset=utf-8"" http..."
...,...,...
8597,[<p>Su número de contacto pf._x000D_\n_x000D_\...,[<p>Su número de contacto pf._x000D_\n_x000D_\...
8598,[<p>-- SOLICITUD DE ENTREVISTA --_x000D_\n_x00...,[<p>-- SOLICITUD DE ENTREVISTA --_x000D_\n_x00...
8599,[<p>Su número de contacto pf._x000D_\n_x000D_\...,[<p>Su número de contacto pf._x000D_\n_x000D_\...
8602,[<p>Muy buenas noches._x000D_\nSaludos para co...,[<p>Muy buenas noches._x000D_\nSaludos para co...


In [43]:
pd.crosstab(master.categorizado_pero_no_matcheado,master.ronda)

ronda,1,2
categorizado_pero_no_matcheado,,
False,4289,4315


In [44]:
len(master[master['categoria'].isnull() & (master.tipo!='solo_publicacion') & (master.ronda==1)])

85

In [45]:
master[master.ronda.isna()]

,ronda,tipo,id_publicacion,link,id,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,to_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,id_pub2,RA_inmob_name,RA_match_comment,time_diff,uno,order,mail_number,categorizado_pero_no_matcheado,categoria,colateral,telefono,automatico,noes,nombre,observacion,ra,from_inmob,subject_inmob,body_inmob


In [46]:
### Abro la base categorizada
ra_new = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\output\categorization\round1-messages April 26\para_categorizar_Agos_15_06_2022.xlsx")

identifica_respuestas  = ['thread_id_inmob','date_inmob_clean','from_inmob']
ra_new = tools.limpia_prueba_google_y_quien_sos(ra_new, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')
# ra_new['fecha'] = tools.limpia_variable_dates(ra_new,['date_inmob'])
ra_new['id_respuesta'] = tools.genera_id(ra_new, identifica_respuestas)


Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 0 de ¿Quien sos?


In [47]:
# Mergeo la base con las clasificaciones de Agos
# ra_new = ra_new.rename(columns={'no es':'noes', 'from':'from_inmob', 'subject':'subject_inmob','body':'body_inmob'})
ra_identifies = ['id_respuesta']
ra_variables = ['categoria','colateral','telefono','automatico','noes','nombre','observacion','ra','from_inmob','subject_inmob','body_inmob']
master = master.merge(ra_new[ra_variables + ra_identifies], on=ra_identifies, how='outer',indicator=True)
print(master._merge.value_counts())

master = tools.normaliza_columnas_post_merge(master,suffixes=('_x','_y')) # VERIFICAR

# Agrego data de la ronda y el tipo para los que no mergearon 
master['ronda'] = np.where(master['_merge']=='right_only',2,master['ronda'])
master['tipo'] = np.where(master['_merge']=='right_only','solo_respuesta',master['tipo'])
master['categorizado_pero_no_matcheado'] = np.where(master['_merge']=='right_only',True,master['categorizado_pero_no_matcheado'])

# Dropeo las columnas irrelevantes
master = master.drop(columns=['_merge'])
master = master[master['categoria']!=0]

left_only     8529
both            75
right_only       0
Name: _merge, dtype: int64
Hay valores diferentes en las columans ra_y y ra_x


,ra_x,ra_y
7679,Alejo,julian


Hay valores diferentes en las columans body_inmob_y y body_inmob_x


,body_inmob_x,body_inmob_y
84,[<p>¡Buenos días! te comento para el ingreso s...,[<p>¡Buenos días! te comento para el ingreso s...
100,"[<p>Estimado, buenas tardes.\r\n\r\nEstán busc...","[<p>Estimado, buenas tardes.\n\nEstán buscando..."
113,[<p>@import url(http://url2214.inmomail.com/ls...,[<p>@import url(http://url2214.inmomail.com/ls...
246,[<p>Su Cliente de Mail NO soporta mensajes en ...,[<p>Su Cliente de Mail NO soporta mensajes en ...
255,[<p>[image: Google]\r\nEl 30 de mayo puede que...,[<p>[image: Google]\nEl 30 de mayo puede que p...
...,...,...
7251,[<p>Hola Pablo Te brindo información de tu in...,[<p>Hola Pablo Te brindo información de tu in...
7319,"[<p>Hola Pablo y Belen,_x000D_\n_x000D_\nEsper...","[<p>Hola Pablo y Belen,\n\nEspero anden muy bi..."
7629,[<p>@import url(http://url2214.inmomail.com/ls...,[<p>@import url(http://url2214.inmomail.com/ls...
7666,[<p>-- SOLICITUD DE ENTREVISTA --_x000D_\n_x00...,[<p>-- SOLICITUD DE ENTREVISTA --\n\n*DATOS DE...


In [48]:
pd.crosstab(master.tipo, master.categoria.isnull())

categoria,False,True
tipo,,
match_ambas,3943,348
solo_publicacion,0,4129
solo_respuesta,149,30


In [ ]:
############### JULI, aca generate las bases para los RA 17/8

In [49]:
master['id_inmobiliaria'] = tools.genera_id(master, ['inmobiliaria','municipio'])
print("Hay", master['id_inmobiliaria'].drop_duplicates().count(), "inmobiliarias diferentes en la base.")

Hay 4364 inmobiliarias diferentes en la base.


In [50]:
pd.crosstab(master.tipo, master.ronda)

ronda,1,2
tipo,,
match_ambas,2117,2174
solo_publicacion,2072,2057
solo_respuesta,95,84


In [51]:
## FIXME: esto no está bien...
pd.crosstab(master.ronda, master.categorizado_pero_no_matcheado)

categorizado_pero_no_matcheado,False
ronda,
1,4284
2,4315


In [52]:
import unicodedata

long_strings = master.columns[master.dtypes == object].to_list()
for string in long_strings:
    master[string] = master[string].astype(str).str[:2040] # Max Stata string length is 2045
    master[string] = master[string].astype(str).apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())

In [53]:
# Exporto las bases para analisis a excel
master.sort_values(by=['ronda','tipo'], inplace=True)
# df.date_inmob_clean = df['date_inmob_clean'].astype(str)
# df.date_conf_clean = df['date_conf_clean'].astype(str)
master = master.loc[:,~master.columns.duplicated()]

master.to_stata(path_dataout + fr"\base_limpia_tarea_2_nico.dta",write_index=False, version=118)

In [54]:
pd.read_stata(path_dataout + fr"\base_limpia_tarea_2_nico.dta")

,ronda,tipo,id_publicacion,link,id,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,to_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,id_pub2,RA_inmob_name,RA_match_comment,time_diff,uno,order,mail_number,categorizado_pero_no_matcheado,categoria,colateral,telefono,automatico,noes,nombre,observacion,ra,from_inmob,subject_inmob,body_inmob,id_inmobiliaria
0,1,match_ambas,29384efa5d2609087db97563147e73a7c0172848af307b...,properati.com.ar/detalle/4a3pz_alquiler_depart...,mmo0159_lp505098,martinez morando,2022-02-14,malvinas_argentinas,buenos aires ampliado,nan,espectaculares lofts a estrenar en el km 40 de...,departamento en alquiler en panamericana km 40...,70000.0,$,1.0,NaN,1.0,NaN,NaN,NaN,70000.0,NaN,NaN,6.0,1.0,6.0,1.0,NaN,1.0,0.0,0.0768,panamericana km 40. 2400,70000.0,martinez morando,hetero,high,Manuel,rodriguez.manuelw@gmail.com,Luis,Florencia,1.0,"Buenos dias, mi nombre es Manuel, junto a mi p...",Manuel y Florencia.,23.0,Hiciste una consulta por la propiedad Panameri...,Properati <contacts@mailing.properati.com.ar>,rodriguez.manuelw@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 16:32:10 +0000 (UTC)",2022-04-19,2022-04-19 13:32:10,arg,70.0,4.5,944cede9eef66ef2acaeab5d48d4368ac466b9ac3ebbfd...,rodriguez.manuelw@gmail.com,19 Apr 2022 13:53:49 -0300,NaN,18042bea6092e389,text/html,arg,2022-04-19,2022-04-19 13:53:49,70.0,4.5,"[Match(start=22, end=37, dist=0, matched='mart...",NaN,0,NaN,nan,NaN,nan,nan,21.650000,1,NaN,NaN,0,6.0,0.0,1.0,0.0,0.0,departamento en alquiler en panamericana km 40...,,Julian,"""Fermin Duenas"" <fermin@martinezmorando.net.ar>",Inmobiliaria consulta,"[<div>\nBuenas tardes,<br/> ...",7de802c19e37f2c95e39e33317a9e92b21ca3fca32302f...
1,1,match_ambas,14cfa790f4d16225b1014b7bd850a66d2d7bda03986cbb...,properati.com.ar/detalle/4eumc_alquiler_depart...,amp0189_lp511908,am propiedades,2022-04-06,caba,buenos aires ampliado,nan,"luminoso monoambiente, con bano completo y coc...",departamento en alquiler en paseo colon 1200,30000.0,$,1.0,NaN,1.0,NaN,NaN,NaN,30000.0,NaN,NaN,21.0,1.0,21.0,1.0,NaN,1.0,1.0,0.0629,paseo colon 1200,30000.0,am,trans,high,Pablo,pablo.gomezfh@gmail.com,Juan,Florencia,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Florencia (ella es trans),23.0,Hiciste una consulta por la propiedad Paseo Co...,Properati <contacts@mailing.properati.com.ar>,pablo.gomezfh@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 20 Apr 2022 13:53:00 +0000 (UTC)",2022-04-20,2022-04-20 10:53:00,arg,90.0,3.1,1846baf3b1e4581b40465797efe0f91aaf761ffc81540b...,pablo.gomezfh@gmail.com,20 Apr 2022 11:20:25 -0300,NaN,180475891aa25c12,multipart/mixed,arg,2022-04-20,2022-04-20 11:20:25,90.0,3.1,"[Match(start=0, end=14, dist=0, matched='am pr...",NaN,0,NaN,nan,NaN,nan,nan,27.416667,1,NaN,NaN,0,6.0,0.0,1.0,NaN,NaN,departamento en alquiler en paseo colon 1200,nan,Agostina,AM Propiedades <aminmobiliaria@hotmail.com>,Detalles de la propiedad,"[<style id=""media-query-bodytag"" type=""text/cs...",b6c6c19a83900750d6cab42a8d01aeda191326d970cf49...
2,1,match_ambas,cd492a6242292d16bc7f13f68fc46effaf9855784218ab...,properati.com.ar/detalle/4ao4m_alquiler_depart...,bah0159_lp506300,gonzalez ferioli personal broker bahia,2022-02-23,tigre,buenos aires ampliado,nan,alquiler temporario a partir de 1 de abril de ...,departamento en alq

In [55]:
master[master['from_inmob'].str.contains('gabriel@interwin.com.ar')]

,ronda,tipo,id_publicacion,link,id,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,to_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,id_pub2,RA_inmob_name,RA_match_comment,time_diff,uno,order,mail_number,categorizado_pero_no_matcheado,categoria,colateral,telefono,automatico,noes,nombre,observacion,ra,from_inmob,subject_inmob,body_inmob,id_inmobiliaria
6997,1,match_ambas,48c84f0d12386f48b36e54e0bc262d83d217d79ff91a51...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",departamento en alquiler en oro al 3000,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333300,1.0,17.0,1.0,16.0,1.0,1.0,2.0,1.0,0.5768,oro 3000,250000.0,interwin marketing,hetero,neutral,Manuel,jf.manuel.rodriguez@gmail.com,Juan,Laura,1.0,"Buenos dias, mi nombre es Manuel, junto a mi p...",Manuel y Laura.,23.0,Hiciste una consulta por la propiedad Oro Al 3000,Properati <contacts@mailing.properati.com.ar>,jf.manuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 20:04:26 +0000 (UTC)",2022-04-19,2022-04-19 17:04:26,arg,NaN,NaN,ce2dc7885cac0f79d97f7883e42a994b13c4552348c118...,jf.manuel.rodriguez@gmail.com,nan,NaN,1804374b86d3e037,nan,nan,NaT,2022-04-19 17:12:43,NaN,NaN,nan,NaN,1,369.0,silvataniaelizabeth@gmail.com,999.0,interwin marketing inmobiliario,nan,8.283333,1,NaN,NaN,False,2.0,0.0,0.0,6.0,0.0,departamento en alquiler en oro al 3000,promo,Julian,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",oro,[<p>sldos_x000D_\n_x000D_\ngabriel_x000D_\n_x0...,16e47ea651005a899657bad13e8b72d4103d9fdcc54bf4...
7866,2,match_ambas,1d7378f53ebbb64aa6117d094445b86a8ed06a0d1ef001...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",nan,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333252,1.0,16.0,1.0,NaN,NaN,1.0,2.0,1.0,NaN,oro 3000,250000.0,interwin marketing,trans,high,Pablo,f.h.pablogomez@gmail.com,Luis,Florencia,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Florencia (ella es trans),2.0,Hiciste una consulta por la propiedad Oro Al 3000,Properati <contacts@mailing.properati.com.ar>,f.h.pablogomez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 04 May 2022 17:29:06 +0000 (UTC)",2022-05-04,2022-05-04 14:29:06,nan,NaN,NaN,30bf331b087a74bac24e02fa96859c920c4a98ae26f43a...,f.h.pablogomez@gmail.com,nan,NaN,180904e64528f81f,nan,nan,NaT,2022-05-04 15:21:36,NaN,NaN,nan,NaN,1,679.0,canuelobelen@gmail.com,NaN,interwin marketing inmobiliario,nan,52.500000,1,NaN,NaN,False,2.0,NaN,NaN,2.0,NaN,departamento en alquiler en oro al 3000,nan,Agostina,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",ALQUILER ORO TODO RECICLADO A NUEVO,"[<p>ALQUILER ORO TODO RECICLADO A NUEVO , EL ...",16e47ea651005a899657bad13e8b72d4103d9fdcc54bf4...
7867,2,match_ambas,1d7378f53ebbb64aa6117d094445b86a8ed06a0d1ef001...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",nan,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333252,1

In [57]:
### Corro dofile de Luis

## Define a Python function to launch a do-file 
def dostata(dofile, cwd, *params):
    import subprocess
    
    cmd = ["C:\Program Files\Stata16\StataMP-64.exe","/e","do", dofile]         
    for param in params:
        cmd.append(param)
    return (subprocess.call(cmd, cwd=cwd))

## Run a do-file
path = path_dropbox + r"\BID-LGBTQ+\campo\code\analisis"
dostata(path + r"\tarea_3_limpieza_base_final.do", cwd=path)


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado